# How to Run
## A guide to getting started with WFP
This Jupyter notebook should give a comprehensive overview for figuring out how to run the imaka wavefront profiler. 

**Contents:**
- Required Python Libraries
- Configuration Files
- Run Files
- Target Files
- Running Correlation Pipeline (Basics)
- Running Estimation Pipeline (Basics) TBD


## => Required Python Libraries
Below is an import list of all functions used throughout this code. If you're able to run the following block of code, you should be able to run this package without issues.

!TODO: check that all libraries are used throughout code

In [1]:
#Pipeline
import os
import sys
import fnmatch
import logging
import threading
import time
import concurrent.futures
#Correlator
import math
import imageio
import pandas as pd
import numpy as np
from scipy.io import readsav
import matplotlib
from astropy.table import Table
from astropy.stats import sigma_clip
from astropy.io import fits
#Plotting
from celluloid import Camera
from IPython.display import Image 

The following imports are from seperate packages that aren't necessary for core functionality:

In [2]:
#datatable: not required for corr code
from datetime import datetime, date, time, timezone
import pytz

In [3]:
# estimator
import hdbscan
import seaborn as sns

## => Configuration Files
Configuration files consist of the directory locations you wish the correlator function to find and store data. 

Location:
`wf_profile/inputs/conf/`

### Defining a variable path: 
name of variable a space, an equals, a space, value, line break. This is the order that the file reader will parse inputs.

> name = value

The file reader for this file will not catch anything that starts with "#" so feel free to use those as comments.

### The major variables to edit: 
The big variables to edit are the paths specific to your system. They are described below:

> `data_path = /home/imaka/data/`

The data path shows the system where to look for files that contain dates. for example, this is the directory where we'd expect to find **/home/imaka/data/**20201102

> `out_path = /home/emcewen/out/`

This is where you'll be storing your data output. The system will create this path if it can, so be sure you're pointing it in the right direction

> `target_path = /home/emcewen/data/target_input/`

The target path shows the system where to look for target files. In this package, target files are listed under: **wf_profile/inputs/targets**


### Varibles that don't need changing:
These are ways to customize how the pipeline operates or what it outputs. Don't feel the need to change them for a typical run. 

> `pipe_fn = all`

This determines whether you want just data fits, plots from previous files, or both at once. **all** should work for most users.

> `parallel = True`

This allows you to do a parallization of the conf run, but is experimental.

In [11]:
# Example configuration file:
file = 'inputs/conf/CONF_data.txt'
print(open(file, 'r').read())

# Configure file for DATA
# => Change data paths here

# Commented lines will be ignored
# variables must be defined with " = " (spaces included)

# where to place files
data_path = /home/imaka/data/
out_path = /home/emcewen/out/
target_path = /home/emcewen/data/target_input/

# options: all, data, plots
pipe_fn = all

# Parallel processing
parallel = True


### Accessing Conf Varibles:
You can access these files with a built in function in `pipeline/code/file_reader.py`.

This is helpful if you ever need to pull from one of these files in your own code building off of the WFS profiler. 

In [13]:
# reading in conf variables
from pipeline.code.file_reader import *

conf_f = 'inputs/conf/CONF_data.txt'
conf_d = read_file_dic(conf_f)

conf_d["data_path"]

'/home/imaka/data/'

## => Run files

Run files are inputs to the main pipeline that outlines either the dates or the files for the correlator to run over. 

Entries can be:
- **a date**, will be looked for in **data_path** and all fits files in date/ or date/ao will be added to a list of files to process
- **target name**, will look for this file in the **target_path** defined in conf, where the file begins with **target_name** and ends with **_USNOB_GS.txt**. If a new target is inserted, all dates after the change will be given that target until switched again. 


In [9]:
# Example Run File: Data
file = 'inputs/runs/RUN10.txt'
print(open(file, 'r').read())

Beehive-W
20190226


In [10]:
# Example Run File: Simulation
file = 'inputs/runs/RUN_simul.txt'
print(open(file, 'r').read())

simul_4GS_12am_6mag
20200904


## => Target files

Target files are optional, but will give more information to analysis. They give information on the targets of each individual WFS in configurations.

**Information included:**
- **file name:** first line (without the target suffix identifier)
- **number of wfs:** how many wfs are specifically used.
- **WFS entries:** WFS number, RA, DEC, and Mag, tab seperated


In [5]:
# Example Run File: Beehive-W

file = open('inputs/targets/Beehive-W_USNOB_GS.txt', 'r')
print (file.read())

Beehive-W
5                      
0	129.961317	19.540806	6.0
3	130.112547	19.544806	6.2
4	130.092028	19.669939	5.9
1	129.927728	19.778453	6.5
2	130.180028	19.719314	6.7


In [6]:
# Example Run File: Simul

file = open('inputs/targets/simul_4GS_12am_6mag_USNOB_GS.txt', 'r')
print (file.read())

simul.4GS.12am.6mag
4
0	6	0	6
1	0	6	6
2	-6	0	6
3	0	-6	6


## => Running Correlation Pipeline (Basics)

Wavefront profiler correlation pipeline Location:

`wf_profile/pipeline/wfp_pipeline`

The correlation pipeline is the main pipeline for this code package. Below we will explain the steps taken with a run. You should have ready the file path for the configuration file you want to use and the run file you'd like to include.

**A pipeline call configuration**

Structure:

> `python3 [complete path to wf_profile] [CONF_file with path] [RUN_file with path]`

Exaple call:

>` python3 pipeline/wf_pipeline inputs/CONF.txt inputs/RUN.txt`

Note that this call is run from the `wf_profile` folder. Be sure to include the paths from your current folder to the conf file and run files that you wish.

Run from the terminal. 


## => Running Estimation Pipeline (Basics)

as of right now, there is no Basic way to use the Estimation pipeline. Please see the corresponding howto document for a walk through example on how to use it. 